In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code
from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter1d
from scipy import stats
import statsmodels.api as sm
from sklearn.neighbors import KernelDensity

In [2]:
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))

In [3]:
set(infos['treatment'].values)

{'25', '25*', 'baits'}

In [9]:
def get_angle(xa,ya,xb,yb):
    dot_product = (xa*xb+ya*yb)/np.sqrt((xa**2+ya**2)*(xb**2+yb**2))
    angle = (np.arccos(dot_product) / (2 * np.pi) * 360)*(1-2*((ya * xb - yb * xa) >= 0))
    return(angle)
def getKernelDensityEstimation(values, x, bandwidth = 0.2, kernel = 'gaussian'):
    model = KernelDensity(kernel = kernel, bandwidth=bandwidth)
    model.fit(values[:, np.newaxis])
    log_density = model.score_samples(x[:, np.newaxis])
    return np.exp(log_density)
def bestBandwidth(data, minBandwidth = 1, maxBandwidth = 20, nb_bandwidths = 30, cv = 30):
    """
    Run a cross validation grid search to identify the optimal bandwidth for the kernel density
    estimation.
    """
    from sklearn.model_selection import GridSearchCV
    model = GridSearchCV(KernelDensity(),
                        {'bandwidth': np.linspace(minBandwidth, maxBandwidth, nb_bandwidths)}, cv=cv) 
    model.fit(data[:, None])
    return model.best_params_['bandwidth']


In [5]:
infos.columns

Index(['plate', 'inst', 'treatment', 'angle', 'curvature', 'density', 'growth',
       'speed', 'straightness', 'dist_P', 'dist_N', 'angle_to_P', 'angle_to_N',
       't', 'hyph', 'x', 'y', 'vx', 'vy', 'xinit', 'yinit', 'density5',
       'grad_density_x5', 'grad_density_y5', 'grad_density_norm5', 'area5',
       'density10', 'grad_density_x10', 'grad_density_y10',
       'grad_density_norm10', 'area10', 'density20', 'grad_density_x20',
       'grad_density_y20', 'grad_density_norm20', 'area20'],
      dtype='object')

In [5]:
infos['abs_angle_to_P']=np.abs(infos['angle_to_P'])
infos['abs_curvature']=np.abs(infos['curvature'])
infos['curvature_scaled']=infos['curvature']*np.sqrt(infos['growth'])
infos['abs_curvature_sq']=np.abs(infos['curvature'])*np.sqrt(infos['growth'])
infos['signed_straight']=(1-infos['straightness'])*(1-2*(infos['angle']<0))
infos['inv_dens']=1/infos['density']
infos['side_cross']=infos.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
infos['x']=infos['x'].astype(float)
infos['vx']=infos['vx'].astype(float)
infos['y']=infos['y'].astype(float)
infos['vy']=infos['vy'].astype(float)
infos['xinit']=infos['xinit'].astype(float)
infos['yinit']=infos['yinit'].astype(float)
blur = 20
infos['v'] = np.sqrt((infos['vx']**2+infos['vy']**2).astype(float))
infos['gd'] = np.sqrt((infos[f'grad_density_x{blur}']**2+infos[f'grad_density_y{blur}']**2).astype(float))
infos['spvgd']=(infos['vx']*infos[f'grad_density_x{blur}']+infos['vy']*infos[f'grad_density_y{blur}'])/(infos['v']*infos['gd'])
infos['angle_vgd']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['vx'],infos['vy'])
infos['angle_vgd2']=get_angle(-infos[f'grad_density_x{blur}'],-infos[f'grad_density_y{blur}'],infos['xinit'],infos['yinit'])
infos['angle_Pgd']=infos['angle_vgd']+infos['angle_to_P']
infos['angle_Ngd']=infos['angle_vgd']+infos['angle_to_N']
# infos['residual']=infos['speed']-f(infos['spvgd'])

In [8]:
infos.to_csv(f'{path_code}/MscThesis/Results/growth_pattern.csv')

In [7]:
infos

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,...,abs_curvature_sq,inv_dens,side_cross,v,gd,spvgd,angle_vgd,angle_vgd2,angle_Pgd,angle_Ngd
0,69,"(40, 153, 190)",25*,-24.883829,-0.010112,0.962319,2460.850743,307.606343,0.979885,39704.557927,...,0.501620,1.039157,None,1397.884831,0.000291,-0.770291,-39.619949,-28.471609,-46.200362,118.265396
1,69,"(40, 153, 190)",25*,38.655532,0.045338,0.869565,852.600387,213.150097,0.968758,37310.115919,...,1.323850,1.150000,None,478.819381,0.000306,-0.715205,-44.340000,-44.258777,-39.815929,128.162629
2,69,"(40, 153, 190)",25*,25.516456,0.043642,1.508696,584.681289,146.170322,0.972825,46687.278602,...,1.055263,0.662824,None,329.734742,0.000288,-0.709634,44.794842,33.970571,-80.205367,43.618046
3,69,"(40, 153, 190)",25*,-2.353297,-0.010463,2.103623,224.914917,55.995415,1.010085,47015.834959,...,0.156916,0.475370,None,131.700418,0.000295,-0.363046,68.712647,67.752613,-83.936721,40.392971
4,69,"(40, 153, 190)",25*,-10.491477,-0.030740,2.376087,341.294258,28.480745,0.937281,47217.737087,...,0.567900,0.420860,None,185.442714,0.000306,-0.386869,67.240183,65.240190,-89.840081,34.037970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13037,424,"(13, 116, 131)",baits,-0.000000,-0.000000,2.710145,76.036890,19.009222,0.925425,46440.456568,...,0.000000,0.368984,left,40.792156,0.000461,0.917025,-156.494931,-158.179615,-70.288738,-177.062436
13038,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,...,1.110343,0.509978,left,272.737969,0.000453,-0.184485,79.368904,63.032934,-55.413237,191.678736
13039,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,...,0.700171,0.343882,left,317.694822,0.000447,0.064463,93.696035,82.589840,-59.410660,190.467970
13040,424,"(13, 116, 131)",baits,5.017723,0.004998,1.029710,1003.890873,250.972718,1.002913,46119.102066,...,0.158367,0.971147,left,583.660860,0.000382,-0.904978,-25.179699,-21.888528,-75.699619,-178.636150


In [6]:
corrected = infos.loc[(infos["straightness"] <= 1)& (infos["speed"] >=25)& (infos["speed"] <400)&(infos["straightness"] > 0.95)&(infos["density"]>0)]
baits = corrected.loc[corrected['treatment']=='baits']
no_baits = corrected .loc[corrected ['treatment']=='25']
no_baits_new = corrected .loc[corrected ['treatment']=='25*']

In [14]:
plt.close('all')
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
# ax.hist(corrected.loc[corrected['treatment'] == '25']['speed'],bins,alpha=0.3,label='Dummy baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['treatment'] == 'baits']['speed'],bins,alpha=0.3,label='P&N baits (heterogeneous rock form)',density=True)
# ax.hist(corrected.loc[corrected['treatment'] == '25*']['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'] == 94]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'] == 69]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'] == 102]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# x = np.linspace(-6,6,100)
x = np.linspace(0,400,100)

# bandwidth = 5
data = corrected.loc[corrected['plate'] == 69]['speed']
# data = corrected.loc[corrected['treatment'] == 'baits']['speed']

cv_bandwidth = bestBandwidth(data)

kde = getKernelDensityEstimation(data, x, bandwidth=cv_bandwidth)
ax.hist(data,30,density=True)

plt.plot(x, kde, alpha = 0.8, label = f'bandwidth = {round(cv_bandwidth, 2)}')
ax.set_xlabel(r'speed ($\mu m.h^{-1} $)')
plt.legend(loc='upper right')
ax.set_ylabel(r'density')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  import sys


Text(0, 0.5, 'density')

In [8]:
from unidip import UniDip
import unidip.dip as dip
data = np.msort(data)
print(dip.diptst(data))
intervals = UniDip(data).run()
print(intervals)

(0.0049828031541715445, 0.8411588411588412, (2327, 3408))
[(0, 3690)]


In [10]:
from unidip import UniDip

# create bi-modal distribution
dat = np.concatenate([np.random.randn(200)-3, np.random.randn(200)+3])

# sort data so returned indices are meaningful
dat = np.msort(dat)

# get start and stop indices of peaks 
intervals = UniDip(dat).run()

In [20]:
right_crossing = [plate for plate in comments.keys() if comments[plate]=='right']
non_turning = [plate for plate in set(corrected['plate'].values) if np.mean(corrected.loc[corrected['plate']==plate]['signed_straight'])<0.005]

In [11]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==94]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = 'density'
ordinate = 'speed'
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N=800
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle between growth direction and gradient of density (°)')

ax.set_ylabel('speed ($\mu m .h^{-1}$)')
ax.set_xlim(0,6)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from scipy.interpolate import interp1d
x = moving_av["spvgd"]
y = moving_av["speed"]
f = interp1d(x, y,copy=True,fill_value=(250,160),bounds_error=False)

In [14]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['angle_vgd2'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False, notch=True,showmeans = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [15]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['signed_straight'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False, notch=True,showmeans = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
tab

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,...,abs_angle_to_P,inv_dens,side_cross,v,gd,spvgd,angle_vgd,angle_vgd2,angle_Pgd,angle_Ngd
0,69,"(40, 153, 190)",25*,-24.883829,-0.010112,0.962319,2460.850743,307.606343,0.979885,39704.557927,...,6.580413,1.039157,None,1397.884831,0.001005,-0.949730,-18.244346,-7.096006,-24.824759,139.640999
1,69,"(40, 153, 190)",25*,38.655532,0.045338,0.869565,852.600387,213.150097,0.968758,37310.115919,...,4.524071,1.150000,None,478.819381,0.001209,-0.986953,-9.265307,-9.184084,-4.741236,163.237323
2,69,"(40, 153, 190)",25*,25.516456,0.043642,1.508696,584.681289,146.170322,0.972825,46687.278602,...,125.000209,0.662824,None,329.734742,0.001330,-0.930811,21.438431,10.614160,-103.561778,20.261635
6,69,"(40, 153, 190)",25*,-10.407711,-0.011552,2.052899,900.982105,225.245526,0.996245,47844.264278,...,73.126647,0.487116,None,520.347000,0.001644,-0.857094,-31.008209,-28.915640,-104.134856,13.155560
7,69,"(40, 153, 190)",25*,-16.336043,-0.022268,1.194928,733.595192,174.665522,0.988511,49581.350710,...,50.872094,0.836871,None,420.386727,0.000845,-0.825177,-34.393538,-25.487273,-85.265631,29.078085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,47451.023860,...,39.741216,1.106656,left,318.190195,0.001332,-0.103831,-84.040167,-85.238858,-44.298951,-146.722969
13036,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,46972.393814,...,35.315072,0.491278,left,389.104099,0.002163,-0.846596,-32.156702,-25.034267,3.158370,-101.492343
13038,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,...,134.782141,0.509978,left,272.737969,0.001703,-0.396878,66.616828,50.280858,-68.165313,178.926660
13039,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,...,153.106695,0.343882,left,317.694822,0.001560,-0.565473,55.564825,44.458630,-97.541870,152.336760


In [39]:
np.mean(tab['angle_vgd2']),np.std(tab['angle_vgd2'])/np.sqrt(len(tab))

(19.21577680475666, 2.0123061891325618)

In [23]:
plt.close('all')
plate_select = corrected.loc[corrected ['plate']==436]
abcisse = 'angle_to_P'
ordinate = 'residual'
tab = baits
baits_sort = tab.sort_values(abcisse)
N=600
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('normalised scalar product speed.density gradient')
ax.set_ylabel('residual($\mu m. h^{-1}$)')
# ax.set_xlim(-0.001,0.001)
# ax.set_ylim(-150,150)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
def get_density_map(plate,t,compress,blur,x,y,fz):
    densities=np.zeros((30000//compress,60000//compress),dtype=np.float)
    select = corrected.loc[(corrected['plate']==plate) & (corrected['t']==t)]
    for index,row in select.iterrows():
        xx = int(row[x])//compress
        yy = int(row[y])//compress
        densities[xx,yy]+=fz(row)
    density_filtered = gaussian_filter(densities,blur)
    return(density_filtered)

    

In [113]:
ts = set(corrected.loc[(corrected['plate']==419)]['t'])
ts = list(ts)
ts.sort()

In [229]:
compress = 100
blur = 20
densities=np.zeros((30000//compress,60000//compress),dtype=np.float)
count=np.zeros((30000//compress,60000//compress),dtype=np.float)

select = corrected
for index,row in select.iterrows():
    xx = int(row['x'])//compress
    yy = int(row['y'])//compress
    densities[xx,yy]+=row['angle_vgd']
    count[xx,yy]+=1
density_filtered = gaussian_filter(densities/(count+(count==0).astype(float)),blur)

In [115]:
densities = [get_density_map(419,t,100,5,'x','y',lambda row : row['speed']) for t in ts]

In [230]:
plt.close('all')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.imshow(density_filtered)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
imageio.mimsave(f'movie.gif', densities,duration = 1)

Lossy conversion from float64 to uint8. Range [0.0, 1.72143677708428]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.957171889756781]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.740454175547317]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.322802597473528]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.498569646579737]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.520259599237698]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 2.4290524530575817]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. R